# Using the QuantumDispatcher Class

A Dispatcher has 3 functions: `run_and_transmit`, `multi_run_and_transmit`, and `batch_run_and_transmit`. 
`run_and_transmit` takes in 3 parameters:
  * pre-operation: a circuit of operations to prepare a transmitted state
  * post-operations: a list operations to run on different devices
  * the number of shots to run the circuit.

`run_and_transmit` is intended to run a single job, while `multi_run_and_transmit` is intended to run multiple job. For `multi_run_and_transmit` the preparation operations are expected to be a list of operations, and the post operations are expected to be a list of lists. The ith element of each list corresponds to the circuit ran as the ith job.
`batch_run_and_transmit` creates and runs all permutations of operations given to it, both pre and post transmission. 

## Example

Consider the scenario for measurement incompatible testing. First, we prepare the BB84 states for Alice, and Bob measures based on the input he gets, y = 0 or y = 1.

So we would like to make 2 measurements, once with Bob's input 0 and another with 1. We can instantaite a `LocalDispatcher` and call the method `multi_run_and_transmit` on this instance to accomplish this. To instantiate a `LocalDispatcher`, we need to feed in one device.

In [2]:
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, IBMQ
# from qiskit.tools.jupyter import *
# from qiskit.visualization import *
# from qiskit.providers.ibmq.managed import IBMQJobManager

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

import numpy as np

import context
from device_independent_test.quantum_communicator import LocalDispatcher

In [8]:
def get_bb84_state():
    qc = QuantumCircuit(4)
    qc.x(1) # create 1
    qc.h(2) # create +
    qc.x(3) # create -
    qc.h(3) # ^
    return qc

def measure_circuit(y):
    qc = QuantumCircuit(4)
    theta = -1.0*(np.pi/4 + 0.5*y*np.pi) # -pi/4 rotation for y=0, -3pi/4 rotation for y=1
    qc.u3(theta,0,0,0)
    qc.measure_all()
    return qc

# initialization for two measurements - both preparing bb84 state
pre_ops = [get_bb84_state(), get_bb84_state()]

# one test sequence with QuantumCircuit(4) + measure_circuit(0)
# and another with QuantumCircuit(4) + measure_circuit(1) 
post_ops = [
    [QuantumCircuit(4), QuantumCircuit(4)],
    [measure_circuit(0), measure_circuit(1)]
]

dispatch = LocalDispatcher([provider.get_backend('ibmq_qasm_simulator')])
counts = dispatch.multi_run_and_transmit(pre_ops, post_ops, 1000)

The function returns the list of counts of each measurement.

In [10]:
counts

[{'1010': 193,
  '1111': 49,
  '0110': 234,
  '0011': 30,
  '0111': 42,
  '1011': 32,
  '0010': 211,
  '1110': 209},
 {'1010': 36,
  '1111': 211,
  '0110': 37,
  '0011': 210,
  '0111': 205,
  '1011': 218,
  '0010': 40,
  '1110': 43}]